In [74]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("Movies Analysis").getOrCreate()

# Φόρτωση του dataset
df = spark.read.json("movie.json")

# Εμφάνιση του σχήματος του DataFrame για να κατανοήσουμε τα δεδομένα
df.printSchema()


root
 |-- _corrupt_record: string (nullable = true)
 |-- actors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- countries: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- directors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genre: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- imdb_url: string (nullable = true)
 |-- img_url: string (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- metascore: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- users_rating: string (nullable = true)
 |-- votes: string (nullable = true)
 |-- year: string (nullable = true)



In [75]:
df.dtypes

[('_corrupt_record', 'string'),
 ('actors', 'array<string>'),
 ('countries', 'array<string>'),
 ('description', 'string'),
 ('directors', 'array<string>'),
 ('genre', 'array<string>'),
 ('imdb_url', 'string'),
 ('img_url', 'string'),
 ('languages', 'array<string>'),
 ('metascore', 'string'),
 ('rating', 'string'),
 ('runtime', 'string'),
 ('tagline', 'string'),
 ('title', 'string'),
 ('users_rating', 'string'),
 ('votes', 'string'),
 ('year', 'string')]

In [76]:
from pyspark.sql.functions import col

# Assuming 'df' is your DataFrame and you want to keep only specific columns
selected_columns = ['metascore', 'users_rating', 'runtime', 'languages', 'genre']

# Selecting the specified columns
df_sel = df.select(selected_columns)



In [77]:
df_sel.dtypes

[('metascore', 'string'),
 ('users_rating', 'string'),
 ('runtime', 'string'),
 ('languages', 'array<string>'),
 ('genre', 'array<string>')]

Data Preparation:
We selected relevant columns for the analysis.
Converted the "metascore" to integer and "users_rating" to float for numerical analysis.
Extracted the "runtime" as an integer value by removing the "min" string and casting it.
Simplified the "genre" and "languages" columns by selecting the first entry in their arrays, assuming it's the primary genre/language.
Handled missing values by filling numerical columns with their mean and categorical columns with a placeholder 
("Unknown" for genre and a created array with "Unknown" for languages).


In [78]:
from pyspark.sql.functions import col, regexp_replace

# Assuming 'df_selected' is your DataFrame
# Convert 'metascore' to IntegerType and 'users_rating' to FloatType
df_sel= df_sel.withColumn("metascore", col("metascore").cast("integer")) \
                         .withColumn("users_rating", col("users_rating").cast("float"))

# Remove " min" from 'runtime' and convert to IntegerType
df_sel= df_sel.withColumn("runtime", regexp_replace("runtime", " min", "").cast("integer"))

# Show the schema of the updated DataFrame to confirm changes
df_sel.printSchema()



root
 |-- metascore: integer (nullable = true)
 |-- users_rating: float (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genre: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [79]:
df_sel.dtypes

[('metascore', 'int'),
 ('users_rating', 'float'),
 ('runtime', 'int'),
 ('languages', 'array<string>'),
 ('genre', 'array<string>')]

In [80]:
# Display the first three rows of the DataFrame
df_sel.show(10)


+---------+------------+-------+--------------------+--------------------+
|metascore|users_rating|runtime|           languages|               genre|
+---------+------------+-------+--------------------+--------------------+
|     null|        null|   null|                null|                null|
|       44|         6.6|     92|           [English]|   [Comedy, Romance]|
|     null|         2.4|     91|           [English]|  [Horror, Thriller]|
|       61|         7.8|    103|   [English, German]|            [Comedy]|
|       73|         7.3|    128|           [English]|   [Crime, Thriller]|
|       66|         7.0|     97|[English, German,...|            [Comedy]|
|       64|         5.3|     87|           [English]|[Fantasy, Horror,...|
|       51|         6.2|    128|  [English, Russian]|[Action, Adventur...|
|       69|         8.0|    143|[English, Russian...|[Action, Adventur...|
|       70|         8.3|    126|           [English]|    [Drama, Romance]|
+---------+------------+-

In [81]:
from pyspark.sql.functions import col

# Assuming 'df_sel' is your DataFrame that contains the 'genre' column as an array of strings
# We select the first element of the array (index 0) for each row
df_sel = df_sel.withColumn("first_genre", col("genre").getItem(0))

# Optionally, to replace the 'genre' column with the 'first_genre' you can rename it back to 'genre'
df_sel = df_sel.drop("genre").withColumnRenamed("first_genre", "genre")

# Show the updated DataFrame structure and the first few rows to verify
df_sel.printSchema()
df_sel.show(10)


root
 |-- metascore: integer (nullable = true)
 |-- users_rating: float (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genre: string (nullable = true)

+---------+------------+-------+--------------------+-------+
|metascore|users_rating|runtime|           languages|  genre|
+---------+------------+-------+--------------------+-------+
|     null|        null|   null|                null|   null|
|       44|         6.6|     92|           [English]| Comedy|
|     null|         2.4|     91|           [English]| Horror|
|       61|         7.8|    103|   [English, German]| Comedy|
|       73|         7.3|    128|           [English]|  Crime|
|       66|         7.0|     97|[English, German,...| Comedy|
|       64|         5.3|     87|           [English]|Fantasy|
|       51|         6.2|    128|  [English, Russian]| Action|
|       69|         8.0|    143|[English, Russian...| Action|


In [82]:
from pyspark.sql.functions import count, when, col

# Counting null values in each column, excluding isnan check for non-numeric columns
null_counts = df_sel.select([count(when(col(c).isNull(), c)).alias(c) for c in df_sel.columns])

null_counts.show()


+---------+------------+-------+---------+-----+
|metascore|users_rating|runtime|languages|genre|
+---------+------------+-------+---------+-----+
|    52449|           4|  12169|        2| 1254|
+---------+------------+-------+---------+-----+



In [83]:
num_observations = df_sel.count()
print(f"Number of observations in the dataset: {num_observations}")


Number of observations in the dataset: 62060


In [84]:
from pyspark.sql.functions import lit, array

# For numerical columns - assuming previous calculation of mean values is correct
df_fil = df_sel.na.fill({
    'metascore': df_sel.agg({'metascore': 'mean'}).first()[0],
    'users_rating': df_sel.agg({'users_rating': 'mean'}).first()[0],
    'runtime': df_sel.agg({'runtime': 'mean'}).first()[0]
})

# For the 'languages' column with array<string> type
df_fil = df_fil.withColumn('languages', when(col('languages').isNull(), array(lit('Unknown'))).otherwise(col('languages')))

# For the 'genre' column which is a simple string
df_fil = df_fil.na.fill({'genre': 'Unknown'})

# Now, df_filled should have no NA values, and the type mismatch error should be resolved


In [85]:
from pyspark.sql.functions import count, when, col

# Counting null values in each column, excluding isnan check for non-numeric columns
null_counts2 = df_fil.select([count(when(col(c).isNull(), c)).alias(c) for c in df_sel.columns])

null_counts2.show()

+---------+------------+-------+---------+-----+
|metascore|users_rating|runtime|languages|genre|
+---------+------------+-------+---------+-----+
|        0|           0|      0|        0|    0|
+---------+------------+-------+---------+-----+



Feature Engineering:
Converted the "genre" and "primary_language" columns from categorical to numerical form using StringIndexer, which assigns an index to each unique category.
Applied OneHotEncoder to these indices to create a binary vector for each category, which is a common practice for handling categorical variables in machine learning.



In [105]:
from pyspark.sql.functions import col

# Εξαγωγή του πρώτου γλώσσας και δημιουργία της στήλης `primary_language`
df_fil = df_fil.withColumn("primary_language", col("languages").getItem(0))


In [106]:
df_fil.printSchema()


root
 |-- metascore: integer (nullable = true)
 |-- users_rating: float (nullable = false)
 |-- runtime: integer (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genre: string (nullable = false)
 |-- primary_language: string (nullable = true)



In [107]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Καταχώρηση των κατηγορικών στηλών
categoricalCols = ["genre", "primary_language"]

# StringIndexer για τη μετατροπή των κατηγορικών στηλών σε αριθμητικούς δείκτες
stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=[x + "Index" for x in categoricalCols], handleInvalid="keep")

# OneHotEncoder για τη μετατροπή των αριθμητικών δεικτών σε one-hot encoded vectors
encoder = OneHotEncoder(inputCols=stringIndexer.getOutputCols(), outputCols=[x + "OHE" for x in categoricalCols])

# Εκπαίδευση του StringIndexerModel
stringIndexerModel = stringIndexer.fit(df_fil)
# Εφαρμογή του μοντέλου στο training set
df_indexed = stringIndexerModel.transform(df_fil)
df_indexed.show()


+---------+------------+-------+--------------------+---------+----------------+----------+---------------------+
|metascore|users_rating|runtime|           languages|    genre|primary_language|genreIndex|primary_languageIndex|
+---------+------------+-------+--------------------+---------+----------------+----------+---------------------+
|       52|    5.814105|     90|           [Unknown]|  Unknown|         Unknown|       7.0|                 58.0|
|       44|         6.6|     92|           [English]|   Comedy|         English|       0.0|                  0.0|
|       52|         2.4|     91|           [English]|   Horror|         English|       3.0|                  0.0|
|       61|         7.8|    103|   [English, German]|   Comedy|         English|       0.0|                  0.0|
|       73|         7.3|    128|           [English]|    Crime|         English|       4.0|                  0.0|
|       66|         7.0|     97|[English, German,...|   Comedy|         English|       0

Modeling:
Assembled the features into a single vector using VectorAssembler, which is necessary for training the machine learning model in Spark MLlib.
Split the dataset into training (75%) and testing (25%) sets to evaluate the model's performance on unseen data.
Trained a Linear Regression model on the training data.
Evaluation:
Evaluated the model using the R-squared metric, which measures the proportion of variance in the dependent variable that is predictable from the independent variables.
Achieved an R-squared value of approximately 0.052, indicating that only about 5.2% of the variance in the metascore can be explained by the model's inputs.
Insights:
The R-squared value suggests that the model has limited predictive power with the selected features. This outcome could be due to several factors, including:

The simplicity of the model: Linear regression might be too basic to capture the complex relationships between the features and the metascore.
Feature selection: Other features not included might have a more significant impact on the metascore. 
Exploring more data or creating new feature interactions might improve model performance.
Data quality: The presence of null values and the approach taken to handle them could affect the model's accuracy.
Additionally, the selection of the first genre/language might oversimplify the dataset's complexity.

In [108]:
from pyspark.sql.functions import col, regexp_replace, lit, array
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator


# Assuming 'df_fil' is your DataFrame prepared previously
# Create 'primary_language' column by extracting the first element from 'languages' array
df_fil = df_fil.withColumn("primary_language", col("languages").getItem(0))

# Use StringIndexer to convert 'genre' and 'primary_language' to numeric indices
genreIndexer = StringIndexer(inputCol="genre", outputCol="genreIndex", handleInvalid="keep")
languageIndexer = StringIndexer(inputCol="primary_language", outputCol="languageIndex", handleInvalid="keep")

# Assembler to combine feature columns into a single features vector
assemblerInputs = ["users_rating", "runtime", "genreIndex", "languageIndex"]
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

# LinearRegression estimator
lr = LinearRegression(featuresCol="features", labelCol="metascore")

# Define the pipeline
pipeline = Pipeline(stages=[genreIndexer, languageIndexer, assembler, lr])

# Split the data
(trainDF, testDF) = df_fil.randomSplit([0.75, 0.25], seed=42)

# Train the model
model = pipeline.fit(trainDF)

# Make predictions
predictions = model.transform(testDF)

# Evaluate the model
evaluator = RegressionEvaluator(labelCol="metascore", predictionCol="prediction", metricName="r2")
r2_score = evaluator.evaluate(predictions)

print(f"R-squared on test data = {r2_score}")

# Diagnostic code: Show some rows to verify transformations
predictions.select("features", "metascore", "prediction").show(5)


R-squared on test data = 0.05201159768279051
+--------------------+---------+------------------+
|            features|metascore|        prediction|
+--------------------+---------+------------------+
|[4.80000019073486...|        8| 50.93596203357873|
|[6.30000019073486...|       12| 52.93628796351618|
|[3.09999990463256...|       13|49.156504879673925|
|[3.70000004768371...|       13| 50.00933506013069|
| [5.0,90.0,10.0,0.0]|       15|51.842892909830425|
+--------------------+---------+------------------+
only showing top 5 rows

